In [0]:
import os
from os import listdir
from os.path import isfile, join

In [0]:
from sklearn.metrics import cohen_kappa_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import numpy as np
import tensorflow as tf
from keras.models import Model
from keras.layers import Dense, Dropout, GlobalAveragePooling2D, BatchNormalization, Activation
from keras.preprocessing.image import load_img, img_to_array, ImageDataGenerator
from keras.optimizers import Adam
from keras.applications import resnet50
from keras.applications import DenseNet121
from keras.utils import to_categorical
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
!ls "/content/drive/My Drive/Colab Notebooks/experiments"

cifar10_cnn.ipynb	dataset
cifar10_resnet50.ipynb	transfer_learing_keras.ipynb


In [0]:
np.random.seed(12)
tf.set_random_seed(12)

In [6]:
parent_dir = '/content/drive/My Drive/Colab Notebooks/experiments/dataset'
sub_dirs = next(os.walk(parent_dir))[1]
print (sub_dirs)

['person', 'bicycle', 'cow', 'bus', 'car', 'bird', 'diningtable', 'cat', 'horse', 'chair', 'dog', 'bottle', 'aeroplane', 'pottedplant', 'boat', 'motorbike']


We have 16 folders containing images from the above mentioned categories.  
Now we will create a dataframe containing category, file_name, and encoding of the catgeory.

In [0]:
files_details = []
for sub_dir in sub_dirs:
    dir_path = join(parent_dir, sub_dir)
    files = [f for f in listdir(dir_path) if isfile(join(dir_path, f))]
    for file in files:
        file_detail = {}
        file_detail['class'] = sub_dir
        file_detail['path'] = join(parent_dir, sub_dir, file)
        if os.path.getsize(file_detail['path'])>0:
            files_details.append(file_detail)
        else:
            pass
        
lbl = LabelEncoder()
df = pd.DataFrame(files_details)
df['class_encoded'] = lbl.fit_transform(df['class'])

In [8]:
df.head()

,class,path,class_encoded
0,person,/content/drive/My Drive/Colab Notebooks/experi...,14
1,person,/content/drive/My Drive/Colab Notebooks/experi...,14
2,person,/content/drive/My Drive/Colab Notebooks/experi...,14
3,person,/content/drive/My Drive/Colab Notebooks/experi...,14
4,person,/content/drive/My Drive/Colab Notebooks/experi...,14


In [9]:
df.shape, df['class'].nunique()

((793, 3), 16)

It is a small dataset. 793 images are present and the number of categories present are 16

In [10]:
df['class'].value_counts()

bus            50
cat            50
horse          50
chair          50
bottle         50
person         50
dog            50
motorbike      50
cow            50
diningtable    50
car            50
aeroplane      50
bicycle        50
bird           50
boat           50
pottedplant    43
Name: class, dtype: int64

The number of images belonging to a category is around 50

In [0]:
# function for getting the image values in the dataframe
def get_np_images(x):
    original_image = load_img(x, target_size=(256, 256))
    numpy_image = img_to_array(original_image)
    #input_image = np.expand_dims(numpy_image, axis=0)
    #processed_image_resnet50 = resnet50.preprocess_input(input_image.copy())
    return numpy_image

In [0]:
df['image'] = df['path'].apply(lambda x : get_np_images(x))

df.head()

In [13]:
resnet_model = resnet50.ResNet50(weights='imagenet', include_top=False, input_shape= (256,256,3))

W0818 15:15:33.066041 140162596165504 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0818 15:15:33.070899 140162596165504 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0818 15:15:33.088472 140162596165504 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4185: The name tf.truncated_normal is deprecated. Please use tf.random.truncated_normal instead.

W0818 15:15:33.131455 140162596165504 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:174: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.

W0818 15:15:33.132346

94658560/94653016 [==============================] - 1s 0us/step


In [0]:
# freeze all the parameters of the resnet model
for layer in resnet_model.layers:
    layer.trainable=False

In [15]:
# add our own layers
x = resnet_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(256)(x) 
x = BatchNormalization()(x)
x = Activation('tanh')(x)
x = Dropout(0.4)(x)
x = Dense(64)(x) 
x = BatchNormalization()(x)
x = Activation('tanh')(x)
x = Dropout(0.4)(x)
preds = Dense(16, activation='softmax')(x)

W0818 15:15:46.035172 140162596165504 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:3445: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [16]:
model=Model(inputs=resnet_model.input,outputs=preds)
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 256, 256, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 262, 262, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 128, 128, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 128, 128, 64) 256         conv1[0][0]                      
__________________________________________________________________________________________________
activation

In [17]:
[(layer, layer.trainable) for layer in model.layers]

[(<keras.engine.input_layer.InputLayer at 0x7f79d1fcd4e0>, False),
 (<keras.layers.convolutional.ZeroPadding2D at 0x7f79d1f0c5f8>, False),
 (<keras.layers.convolutional.Conv2D at 0x7f79d1f0c3c8>, False),
 (<keras.layers.normalization.BatchNormalization at 0x7f79d1f0c8d0>, False),
 (<keras.layers.core.Activation at 0x7f79d1f0cb00>, False),
 (<keras.layers.convolutional.ZeroPadding2D at 0x7f79c4017c18>, False),
 (<keras.layers.pooling.MaxPooling2D at 0x7f79d1e94ef0>, False),
 (<keras.layers.convolutional.Conv2D at 0x7f79d1eca780>, False),
 (<keras.layers.normalization.BatchNormalization at 0x7f79c0fe9c88>, False),
 (<keras.layers.core.Activation at 0x7f79c0fe9e48>, False),
 (<keras.layers.convolutional.Conv2D at 0x7f79c0f596a0>, False),
 (<keras.layers.normalization.BatchNormalization at 0x7f79c0fbce10>, False),
 (<keras.layers.core.Activation at 0x7f79c0f7b7b8>, False),
 (<keras.layers.convolutional.Conv2D at 0x7f79c0e45eb8>, False),
 (<keras.layers.convolutional.Conv2D at 0x7f79c0e64b3

In [18]:
# divide the dataframe into training and validation dataframe
df = df.sample(frac=1, random_state=12).reset_index(drop=True)
train, valid = train_test_split(df, test_size=0.2, random_state = 12)
train.reset_index(drop = True, inplace = True)
valid.reset_index(drop = True, inplace = True)
train.shape, valid.shape

((634, 4), (159, 4))

In [0]:
# divide into x and y train and test
X_train, y_train = train['image'], train['class_encoded']
X_test, y_test = valid['image'], valid['class_encoded']

# make the Pandas series into numpy ndarray
X_train = np.array([image for image in X_train])
X_test = np.array([image for image in X_test])

# preprocess the images for feeding into the resnet model
X_train = resnet50.preprocess_input(X_train)
X_test = resnet50.preprocess_input(X_test)

# one hot encode the target variables
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

In [20]:
opt = Adam(lr=0.0001)
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

W0818 15:15:47.054345 140162596165504 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.



In [21]:
model.fit(X_train, y_train, validation_split=0.2, epochs=10, batch_size = 32)

W0818 15:15:47.181389 140162596165504 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 507 samples, validate on 127 samples
Epoch 1/10
507/507 [==============================] - 12s 23ms/step - loss: 3.0355 - acc: 0.0966 - val_loss: 2.6278 - val_acc: 0.1654
Epoch 2/10
507/507 [==============================] - 3s 6ms/step - loss: 2.5346 - acc: 0.1992 - val_loss: 2.3185 - val_acc: 0.2756
Epoch 3/10
507/507 [==============================] - 3s 6ms/step - loss: 2.1729 - acc: 0.3116 - val_loss: 2.0907 - val_acc: 0.3701
Epoch 4/10
507/507 [==============================] - 3s 6ms/step - loss: 1.9147 - acc: 0.3767 - val_loss: 1.9382 - val_acc: 0.4094
Epoch 5/10
507/507 [==============================] - 3s 6ms/step - loss: 1.7713 - acc: 0.4497 - val_loss: 1.8333 - val_acc: 0.4567
Epoch 6/10
507/507 [==============================] - 3s 6ms/step - loss: 1.6449 - acc: 0.4970 - val_loss: 1.7468 - val_acc: 0.4724
Epoch 7/10
507/507 [==============================] - 3s 6ms/step - loss: 1.5250 - acc: 0.5621 - val_loss: 1.6782 - val_acc: 0.5276
Epoch 8/10
507/507 [========

In [22]:
model.evaluate(X_test, y_test)

159/159 [==============================] - 1s 5ms/step


[1.433834081175942, 0.578616349577154]

Data Augmentation

In [0]:
# divide into x and y train and test
X_train, y_train = train['image'], train['class_encoded']
X_test, y_test = valid['image'], valid['class_encoded']

# make the Pandas series into numpy ndarray
X_train = np.array([image for image in X_train])
X_test = np.array([image for image in X_test])

X_train_n, X_valid, y_train_n, y_valid = train_test_split(X_train, y_train, test_size=0.2, random_state = 12)

# preprocess the images for feeding into the resnet model
X_train_p = resnet50.preprocess_input(X_train_n)
X_valid_p = resnet50.preprocess_input(X_valid)
X_test_p = resnet50.preprocess_input(X_test)

# one hot encode the target variables
y_train_c = to_categorical(y_train_n)
y_valid_c = to_categorical(y_valid)
y_test_c = to_categorical(y_test)

In [24]:
X_train_p.shape, X_valid_p.shape, y_train_c.shape, y_valid_c.shape

((507, 256, 256, 3), (127, 256, 256, 3), (507, 16), (127, 16))

In [0]:
datagen = ImageDataGenerator(
        width_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest')

In [0]:
datagen.fit(X_train_p)

In [28]:
model.fit_generator(datagen.flow(X_train_p, y_train_c,
                    batch_size=32),
                    samples_per_epoch=X_train_n.shape[0],
                    nb_epoch=30,
                    validation_data=(X_valid_p, y_valid_c))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
  """
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras_pre..., validation_data=(array([[[..., steps_per_epoch=15, epochs=30)`
  """


Epoch 1/30
15/15 [==============================] - 7s 482ms/step - loss: 1.4873 - acc: 0.5771 - val_loss: 0.9999 - val_acc: 0.7323
Epoch 2/30
15/15 [==============================] - 7s 439ms/step - loss: 1.4045 - acc: 0.5770 - val_loss: 0.9777 - val_acc: 0.7480
Epoch 3/30
15/15 [==============================] - 7s 441ms/step - loss: 1.4004 - acc: 0.5679 - val_loss: 0.9630 - val_acc: 0.7480
Epoch 4/30
15/15 [==============================] - 7s 441ms/step - loss: 1.2983 - acc: 0.6496 - val_loss: 0.9498 - val_acc: 0.7559
Epoch 5/30
15/15 [==============================] - 7s 448ms/step - loss: 1.2693 - acc: 0.6615 - val_loss: 0.9526 - val_acc: 0.7480
Epoch 6/30
15/15 [==============================] - 7s 451ms/step - loss: 1.2036 - acc: 0.6528 - val_loss: 0.9444 - val_acc: 0.7480
Epoch 7/30
15/15 [==============================] - 7s 443ms/step - loss: 1.2206 - acc: 0.6559 - val_loss: 0.9410 - val_acc: 0.7480
Epoch 8/30
15/15 [==============================] - 7s 444ms/step - loss: 1.

In [29]:
model.evaluate(X_test_p, y_test_c)

159/159 [==============================] - 1s 6ms/step


[1.0787548599003245, 0.7106918261486029]

In [41]:
densenet_model = DenseNet121(weights='imagenet', include_top=False, input_shape= (256,256,3))

29089792/29084464 [==============================] - 1s 0us/step


In [0]:
# freeze all the parameters of the densenet model
for layer in resnet_model.layers:
    layer.trainable=False

In [0]:
# add our own layers
x = densenet_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(256)(x) 
x = BatchNormalization()(x)
x = Activation('tanh')(x)
x = Dropout(0.4)(x)
x = Dense(64)(x) 
x = BatchNormalization()(x)
x = Activation('tanh')(x)
x = Dropout(0.4)(x)
preds = Dense(16, activation='softmax')(x)

In [0]:
model=Model(inputs=densenet_model.input,outputs=preds)
opt = Adam(lr=0.0001)
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

In [45]:
model.fit_generator(datagen.flow(X_train_p, y_train_c,
                    batch_size=32),
                    samples_per_epoch=X_train_n.shape[0],
                    nb_epoch=30,
                    validation_data=(X_valid_p, y_valid_c))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
  """
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras_pre..., validation_data=(array([[[..., steps_per_epoch=15, epochs=30)`
  """


Epoch 1/30
15/15 [==============================] - 43s 3s/step - loss: 2.8590 - acc: 0.1187 - val_loss: 2.2865 - val_acc: 0.2598
Epoch 2/30
15/15 [==============================] - 9s 626ms/step - loss: 2.2396 - acc: 0.2891 - val_loss: 1.9035 - val_acc: 0.4488
Epoch 3/30
15/15 [==============================] - 9s 603ms/step - loss: 1.8218 - acc: 0.4376 - val_loss: 1.7124 - val_acc: 0.5354
Epoch 4/30
15/15 [==============================] - 9s 608ms/step - loss: 1.5330 - acc: 0.5287 - val_loss: 1.6127 - val_acc: 0.5276
Epoch 5/30
15/15 [==============================] - 9s 605ms/step - loss: 1.3076 - acc: 0.6621 - val_loss: 1.5246 - val_acc: 0.5591
Epoch 6/30
15/15 [==============================] - 9s 602ms/step - loss: 1.1666 - acc: 0.7139 - val_loss: 1.4279 - val_acc: 0.6142
Epoch 7/30
15/15 [==============================] - 9s 598ms/step - loss: 1.0635 - acc: 0.7574 - val_loss: 1.3899 - val_acc: 0.6220
Epoch 8/30
15/15 [==============================] - 9s 595ms/step - loss: 0.91

In [46]:
model.evaluate(X_test_p, y_test_c)

159/159 [==============================] - 1s 6ms/step


[1.2552901004095498, 0.6226415120580662]